In [1]:
##Convert to a three-channel image with only green
import cv2 as cv
import numpy as np
import glob 
img_files=glob.glob('C:\\Users\\Administrator\\Desktop\\weixiaojinrun6\\mpm\\*.tif')
for num,img_file in enumerate(img_files):
    img=cv.imread(img_file)
    blue,green,red=cv.split(img)
    red=np.zeros_like(red)
    blue=np.zeros_like(blue)
    img_new=cv.merge([blue,green,red])
    file_name=os.path.basename(img_file).split('.')[0]
    cv.imwrite('C:\\Users\\Administrator\\Desktop\\weixiaojinrun6\\SHG\\'+str(file_name)+'.tif',img_new)

In [2]:
##Convert to a three-channel image with only red color
import cv2 as cv
import numpy as np
import glob 
img_files=glob.glob('C:\\Users\\Administrator\\Desktop\\weixiaojinrun6\\MPM\\*.tif')
for num,img_file in enumerate(img_files):
    img=cv.imread(img_file)
    blue,green,red=cv.split(img)
    green=np.zeros_like(green)
    blue=np.zeros_like(blue)
    img_new=cv.merge([blue,green,red])
    file_name=os.path.basename(img_file).split('.')[0]
    cv.imwrite('C:\\Users\\Administrator\\Desktop\\weixiaojinrun6\\TPEF\\'+str(file_name)+'.tif',img_new)

In [10]:
#Fusion of cells and MPM images
import scipy.io as sio
import numpy as np
import cv2 as cv
from PIL import Image
import glob 
import os
def hover_net_cell_and_mpm_interation(mat_path,original_image_path,only_cell_path,mask_path,mpm_path,fusion_path):
    #mat_path is a mat file generated by hover-net
    #The premise is that the original HE image must be put into the original_image_path, and all the original HE files must be TIF files
    #only_cell_path is the path of the generated cell image
    #mask_path is the path of the generated mask image
    #The premise is that the MPM image must be put into the original_image_path, and all the original MPM files must be TIF files
    #fusion_path is the path of generated fusion image
    if not os.path.exists(only_cell_path):
        os.makedirs(only_cell_path)
    if not os.path.exists(fusion_path):
        os.makedirs(fusion_path)
    if not os.path.exists(mask_path):
        os.makedirs(mask_path)
    mat_paths=glob.glob(mat_path+"*.mat")
    for num,path in enumerate(mat_paths):
        filename=os.path.splitext(path)[0]
        mat=sio.loadmat(path)
        i1=np.array(mat['inst_map'])
        i1[i1>=1]=1
        i1=i1[...,np.newaxis]
        i2=i1
        i3=np.concatenate((i1,i2),axis=-1)
        i4=np.concatenate((i3,i1),axis=-1)
        cv.imwrite(os.path.join(mask_path,os.path.basename(filename))+".tif",(i4*255).astype(np.uint8))
        a=cv.imread(os.path.join(original_image_path,os.path.basename(filename))+".tif") 
        print(a.shape)
        print(i4.shape)
        result=np.multiply(a,i4)
        result=np.uint8(result)
        mpm=cv.imread(os.path.join(mpm_path,os.path.basename(filename))+".tif")
        result1=np.array(result)+np.array(mpm)
        result1=np.clip(result1,0,255)
        cv.imwrite(os.path.join(fusion_path,os.path.basename(filename))+".tif",result1,((int(cv.IMWRITE_TIFF_RESUNIT), 2,
                                                                  int(cv.IMWRITE_TIFF_COMPRESSION), 1,
                                                                  int(cv.IMWRITE_TIFF_XDPI), 96,
                                                                  int(cv.IMWRITE_TIFF_YDPI), 96)))
        result[result==0]=255
        cv.imwrite(os.path.join(only_cell_path,os.path.basename(filename))+".tif",result,((int(cv.IMWRITE_TIFF_RESUNIT), 2,
                                                                  int(cv.IMWRITE_TIFF_COMPRESSION), 1,
                                                                  int(cv.IMWRITE_TIFF_XDPI), 96,
                                                                  int(cv.IMWRITE_TIFF_YDPI), 96)))


In [12]:
hover_net_cell_and_mpm_interation('F:\\microinvasion\\dataset\\pred\\mat\\',
'F:\\microinvasion\\dataset\\imgs\\he',
'F:\\microinvasion\\dataset\\imgs\\only-cell\\',
'F:\\microinvasion\\dataset\\imgs\\mask\\',
'F:\\microinvasion\\dataset\\imgs\\mpm\\',
'F:\\microinvasion\\dataset\\imgs\\fusion\\')

(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
